<a href="https://colab.research.google.com/github/SunkaraboinaPraveenKumar/Machine_Learning_Projects/blob/main/GPT2_Disease_Symptoms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchtext transformers sentencepiece pandas tqdm datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset, DatasetDict,Dataset
import pandas as pd
import ast
import datasets
import tqdm as tqdm
import time

In [3]:
data_sample=load_dataset("QuyenAnhDE/Diseases_Symptoms")

README.md:   0%|          | 0.00/381 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


Diseases_Symptoms.csv:   0%|          | 0.00/107k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/400 [00:00<?, ? examples/s]

In [4]:
data_sample

DatasetDict({
    train: Dataset({
        features: ['Code', 'Name', 'Symptoms', 'Treatments'],
        num_rows: 400
    })
})

In [5]:
!pip install torch torchtext transformers sentencepiece pandas tqdm datasets

In [6]:
from datasets import load_dataset,DatasetDict,Dataset
import pandas as pd
import ast
import datasets
import tqdm as tqdm
import time

data_sample = load_dataset("QuyenAnhDE/Diseases_Symptoms")

Repo card metadata block was not found. Setting CardData to empty.


In [7]:
data_sample

DatasetDict({
    train: Dataset({
        features: ['Code', 'Name', 'Symptoms', 'Treatments'],
        num_rows: 400
    })
})

In [8]:
updated_data=[{'Name':item['Name'],'Symptoms':item['Symptoms']} for item in data_sample['train']]

In [9]:
df=pd.DataFrame(updated_data)
df.head()

,Name,Symptoms
0,Panic disorder,"Palpitations, Sweating, Trembling, Shortness o..."
1,Vocal cord polyp,"Hoarseness, Vocal Changes, Vocal Fatigue"
2,Turner syndrome,"Short stature, Gonadal dysgenesis, Webbed neck..."
3,Cryptorchidism,"Absence or undescended testicle(s), empty scro..."
4,Ethylene glycol poisoning-1,"Nausea, vomiting, abdominal pain, General mala..."


In [10]:
# just extract symptoms
df['Symptoms']=df['Symptoms'].apply(lambda x:', '.join(x.split(', ')))

In [11]:
df.head()

,Name,Symptoms
0,Panic disorder,"Palpitations, Sweating, Trembling, Shortness o..."
1,Vocal cord polyp,"Hoarseness, Vocal Changes, Vocal Fatigue"
2,Turner syndrome,"Short stature, Gonadal dysgenesis, Webbed neck..."
3,Cryptorchidism,"Absence or undescended testicle(s), empty scro..."
4,Ethylene glycol poisoning-1,"Nausea, vomiting, abdominal pain, General mala..."


In [12]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [13]:
if torch.cuda.is_available():
    device=torch.device('cuda')
else:
  device=torch.device('cpu')

In [14]:
device

device(type='cuda')

In [15]:
tokenizer=GPT2Tokenizer.from_pretrained('distilgpt2')
model = GPT2LMHeadModel.from_pretrained('distilgpt2').to(device)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [16]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [17]:
BATCH_SIZE=8

In [18]:
df.describe()

,Name,Symptoms
count,400,400
unique,392,395
top,Sciatica,"Swelling, pain, dry mouth, bad taste"
freq,3,3


In [19]:
# Data Set Prep
class LanguageDataset(Dataset):
  def __init__(self,df,tokenizer):
    self.labels=df.columns
    self.data=df.to_dict(orient='records')
    self.tokenizer=tokenizer
    x= self.fittest_max_length(df)
    self.max_length=x
  def __len__(self):
    return len(self.data)

  def __getitem__(self,idx):
    x=self.data[idx][self.labels[0]]
    y=self.data[idx][self.labels[1]]
    text=f"{x} | {y}"
    tokens=self.tokenizer.encode_plus(text,max_length=128,padding='max_length',truncation=True,return_tensors='pt')
    return tokens

  def fittest_max_length(self,df):
    max_length=max(len(max(df[self.labels[0]],key=len)),len(max(df[self.labels[1]],key=len)))
    x=2
    while x<max_length:
      x=x*2

    return x

In [20]:
data_sample=LanguageDataset(df,tokenizer)

In [21]:
data_sample

In [22]:
train_size=int(0.8*len(data_sample))
val_size=len(data_sample)-train_size

train_data,val_data=random_split(data_sample,[train_size,val_size])

In [23]:
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False)


In [24]:
num_epochs=8
learning_rate=5e-4
batch_size=BATCH_SIZE
model_name="distilgpt2"
gpu=0

In [25]:
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
tokenizer.pad_token = tokenizer.eos_token

In [26]:
results=pd.DataFrame(columns=['epoch','transfomer','batch_size','gpu','train_loss','val_loss','epoch_duration_sec'])


In [29]:
import time
import pandas as pd
from tqdm import tqdm  # Ensure tqdm is imported correctly

# Assuming other necessary imports like PyTorch and model setup are done

# Training loop
for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    epoch_training_loss = 0
    train_iterator = tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs} Batch Size: {batch_size} GPU: {gpu} Transformer: {model_name}")

    for batch in train_iterator:
        optimizer.zero_grad()
        inputs = batch['input_ids'].squeeze(1).to(device)
        targets = inputs.clone()
        outputs = model(input_ids=inputs, labels=targets)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        train_iterator.set_postfix({'Training loss': loss.item()})
        epoch_training_loss += loss.item()

    avg_epoch_training_loss = epoch_training_loss / len(train_iterator)

    model.eval()
    epoch_val_loss = 0
    val_iterator = tqdm(val_loader, desc=f"Validating epoch {epoch+1}/{num_epochs}")

    with torch.no_grad():
        for batch in val_iterator:
            inputs = batch['input_ids'].squeeze(1).to(device)
            targets = inputs.clone()
            outputs = model(input_ids=inputs, labels=targets)
            loss = outputs.loss
            epoch_val_loss += loss.item()
            val_iterator.set_postfix({'Validation loss': loss.item()})

    avg_epoch_val_loss = epoch_val_loss / len(val_iterator)
    end_time = time.time()
    epoch_duration_sec = end_time - start_time

    new_row = {
        'transformer': model_name,
        'batch_size': batch_size,
        'gpu': gpu,
        'epoch': epoch + 1,
        'train_loss': avg_epoch_training_loss,
        'val_loss': avg_epoch_val_loss,
        'epoch_duration_sec': epoch_duration_sec  # Fixed typo here
    }

    results = pd.concat([results, pd.DataFrame([new_row])], ignore_index=True)
    results.to_csv('results.csv', index=False)
    print(f"Epoch: {epoch+1}, Validation Loss: {avg_epoch_val_loss}")

Training Epoch 1/8 Batch Size: 8 GPU: 0 Transformer: distilgpt2: 100%|██████████| 40/40 [00:08<00:00,  4.63it/s, Training loss=0.562]
Validating epoch 1/8: 100%|██████████| 10/10 [00:00<00:00, 18.91it/s, Validation loss=0.65]
<ipython-input-29-a566194a7004>:54: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, pd.DataFrame([new_row])], ignore_index=True)


Epoch: 1, Validation Loss: 0.700450137257576


Training Epoch 2/8 Batch Size: 8 GPU: 0 Transformer: distilgpt2: 100%|██████████| 40/40 [00:07<00:00,  5.68it/s, Training loss=0.687]
Validating epoch 2/8: 100%|██████████| 10/10 [00:00<00:00, 18.91it/s, Validation loss=0.607]


Epoch: 2, Validation Loss: 0.6829866170883179


Training Epoch 3/8 Batch Size: 8 GPU: 0 Transformer: distilgpt2: 100%|██████████| 40/40 [00:07<00:00,  5.61it/s, Training loss=0.34]
Validating epoch 3/8: 100%|██████████| 10/10 [00:00<00:00, 19.25it/s, Validation loss=0.643]


...

Epoch: 6, Validation Loss: 0.8139260649681092


Training Epoch 7/8 Batch Size: 8 GPU: 0 Transformer: distilgpt2: 100%|██████████| 40/40 [00:07<00:00,  5.57it/s, Training loss=0.129]
Validating epoch 7/8: 100%|██████████| 10/10 [00:00<00:00, 18.74it/s, Validation loss=0.846]


Epoch: 7, Validation Loss: 0.8786725997924805


Training Epoch 8/8 Batch Size: 8 GPU: 0 Transformer: distilgpt2: 100%|██████████| 40/40 [00:07<00:00,  5.56it/s, Training loss=0.0927]
Validating epoch 8/8: 100%|██████████| 10/10 [00:00<00:00, 18.82it/s, Validation loss=0.906]

Epoch: 8, Validation Loss: 0.9400659024715423


In [40]:
inputs_str="Pneumonia"
inputs_ids=tokenizer.encode_plus(inputs_str,return_tensors='pt').to(device)


In [41]:
inputs_ids

{'input_ids': tensor([[   47, 25668, 11339]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1]], device='cuda:0')}

In [42]:
output = model.generate(
    input_ids=inputs_ids['input_ids'],
    attention_mask=inputs_ids['attention_mask'],
    max_length=100,
    num_return_sequences=1,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.5,
    repetition_penalty=1.2,
    length_penalty=1.0,
    pad_token_id=tokenizer.eos_token_id
)

In [43]:
output

tensor([[   47, 25668, 11339,   930,   399,   682,    64,    11, 30874,    11,
         36004,    11, 32692,  2356,   393,  1067,  9430,    11,  5801,  2612,
          2494, 50256]], device='cuda:0')

In [44]:
decoded_output=tokenizer.decode(output[0],skip_special_tokens=True)

In [45]:
decoded_output

'Pneumonia | Nausea, vomiting, diarrhea, abdominal pain or cramps, rapid heart rate'